<a href="https://colab.research.google.com/github/open-mmlab/mmselfsup/blob/master/demo/mmselfsup_colab_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MMSelfSup Tutorial
In this tutorial, we will introduce the following content:

- How to install MMSelfSup
- How to train algorithms in MMSelfSup
- How to train downstream tasks

If you have any other questions, welcome to report issues.

## How to install MMSelfSup

Before using MMSelfSup, we need to prepare the environment with the following steps:

1. Install Python, CUDA, C/C++ compiler and git
2. Install PyTorch (CUDA version)
3. Install dependent codebase (mmcv, mmcls)
4. Clone mmselfsup source code from GitHub and install it

Because this tutorial is on Google Colab and all necessary packages have been installed, we can skip the first two steps.

In [1]:
!pwd

/content


In [2]:
# Check nvcc version
!nvcc -V

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Mon_Oct_12_20:09:46_PDT_2020
Cuda compilation tools, release 11.1, V11.1.105
Build cuda_11.1.TC455_06.29190527_0


In [3]:
# Check GCC version
!gcc --version

gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [4]:
# Check PyTorch installation
import torch, torchvision
print(torch.__version__)
print(torch.cuda.is_available())

1.10.0+cu111
True


## Install MMCV

MMCV is the basic package of all OpenMMLab packages. We have pre-built wheels on Linux, so we can download and install them directly.

Please pay attention to PyTorch and CUDA versions to match the wheel.

In the above steps, we have checked the version of PyTorch and CUDA, and they are 1.10.2 and 11.3 respectively, so we need to choose the corresponding wheel.

In addition, we can also install the full version of mmcv (mmcv-full). It includes full features and various CUDA ops out of the box, but needs a longer time to build.

MIM is recommended: https://github.com/open-mmlab/mim

In [5]:
!pip install openmim

  Created wheel for openmim: filename=openmim-0.1.5-py2.py3-none-any.whl size=42503 sha256=6957857625db07e7d2c995d0fa19d98dd58f3daea577a3bc933b16edab669fb4
  Stored in directory: /root/.cache/pip/wheels/16/8b/e1/bdebbbc687aa50224a5ce46fe97a040a0c59f92b34bfc750b6
Successfully built openmim


In [6]:
!mim install mmcv-full

installing mmcv-full from wheel.
Looking in links: https://download.openmmlab.com/mmcv/dist/cu111/torch1.10.0/index.html
     |████████████████████████████████| 46.3 MB 158 kB/s 
     |████████████████████████████████| 190 kB 3.9 MB/s 


Besides, you can also use pip to install the packages, but you are supposed to check the pytorch and cuda version manually. The example command is provided below, but you need to modify it according to your PyTorch and CUDA version.

In [ ]:
# Install mmcv and mmcls
!pip install mmcv-full -f https://download.openmmlab.com/mmcv/dist/cu111/torch1.10/index.html

## Clone and install mmselfsup

In [7]:
# Clone MMSelfSup repository
!git clone https://github.com/open-mmlab/mmselfsup.git
%cd mmselfsup/

# Install MMSelfSup from source
!pip install -e . 

Cloning into 'mmselfsup'...
remote: Enumerating objects: 3255, done.
remote: Counting objects: 100% (872/872), done.
remote: Compressing objects: 100% (524/524), done.
remote: Total 3255 (delta 534), reused 446 (delta 347), pack-reused 2383
Receiving objects: 100% (3255/3255), 1.96 MiB | 6.78 MiB/s, done.
Resolving deltas: 100% (1911/1911), done.
/content/mmselfsup
Obtaining file:///content/mmselfsup
     |████████████████████████████████| 490 kB 4.3 MB/s 
     |████████████████████████████████| 431 kB 48.8 MB/s 
  Running setup.py develop for mmselfsup


In [8]:
# Check MMSelfSup installation
import mmselfsup
print(mmselfsup.__version__)

0.7.1


## Example to start a self-supervised task

Before you start training, you need to prepare your dataset, please check [prepare_data.md](https://github.com/open-mmlab/mmselfsup/blob/master/docs/en/prepare_data.md) file carefully.

**Note**: As we follow the original algorithms to implement our codes, so many algorithms are supposed to run on distributed mode, they are not supported on 1 GPU training officially. You can check it [here](https://github.com/open-mmlab/mmselfsup/blob/master/tools/train.py#L120).


In [9]:
!pwd

/content/mmselfsup


Here we provide a example and download a small dataset to display the demo.

In [10]:
!mkdir data
!wget https://download.openmmlab.com/mmselfsup/data/imagenet_examples.zip
!unzip -q imagenet_examples.zip -d ./data/

--2022-03-25 05:21:14--  https://download.openmmlab.com/mmselfsup/data/imagenet_examples.zip
Resolving download.openmmlab.com (download.openmmlab.com)... 47.75.20.18
Connecting to download.openmmlab.com (download.openmmlab.com)|47.75.20.18|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 155496559 (148M) [application/zip]
Saving to: ‘imagenet_examples.zip’

imagenet_examples.z 100%[===================>] 148.29M  14.2MB/s    in 12s     

2022-03-25 05:21:26 (12.5 MB/s) - ‘imagenet_examples.zip’ saved [155496559/155496559]



In [11]:
# Check data directory
!apt-get install tree
!tree -d ./data

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  tree
0 upgraded, 1 newly installed, 0 to remove and 39 not upgraded.
Need to get 40.7 kB of archives.
After this operation, 105 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tree amd64 1.7.0-5 [40.7 kB]
Fetched 40.7 kB in 1s (45.2 kB/s)
Selecting previously unselected package tree.
(Reading database ... 156210 files and directories currently installed.)
Preparing to unpack .../tree_1.7.0-5_amd64.deb ...
Unpacking tree (1.7.0-5) ...
Setting up tree (1.7.0-5) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
./data
└── imagenet
    ├── meta
    └── train
        └── n01440764

4 directories


### Create a new config file
To reuse the common parts of different config files, we support inheriting multiple base config files. For example, to train `relative_loc` algorithm, the new config file can create the model's basic structure by inheriting `configs/_base_/models/relative-loc.py`.

In [12]:
%%writefile configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py
_base_ = [
    '../_base_/models/relative-loc.py',
    '../_base_/datasets/imagenet_relative-loc.py',
    '../_base_/schedules/sgd_steplr-200e_in1k.py',
    '../_base_/default_runtime.py',
]

log_config = dict(interval=10)

# optimizer
optimizer = dict(
    type='SGD',
    lr=0.2,
    weight_decay=1e-4,
    momentum=0.9,
    paramwise_options={
        '\\Aneck.': dict(weight_decay=5e-4),
        '\\Ahead.': dict(weight_decay=5e-4)
    })

# learning policy
lr_config = dict(policy='step', step=[1])

# runtime settings
runner = dict(type='EpochBasedRunner', max_epochs=70)
# the max_keep_ckpts controls the max number of ckpt file in your work_dirs
# if it is 3, when CheckpointHook (in mmcv) saves the 4th ckpt
# it will remove the oldest one to keep the number of total ckpts as 3
checkpoint_config = dict(interval=1, max_keep_ckpts=3)


Writing configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py


### Read the config file and modify config

We can modify the loaded config file.

In [13]:
# Load the basic config file
from mmcv import Config
cfg = Config.fromfile('configs/selfsup/relative_loc/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab.py')

# Specify the data settings
cfg.data.samples_per_gpu = 64
cfg.data.workers_per_gpu = 2

# Modify the path and meta files of validation dataset
cfg.data.val.data_source.data_prefix = 'data/imagenet/train'
cfg.data.val.data_source.ann_file = 'data/imagenet/meta/train.txt'

# Specify the optimizer
cfg.optimizer = dict(type='SGD', lr=0.005, momentum=0.9, weight_decay=0.0001)
cfg.optimizer_config = dict(grad_clip=None)

# Specify the learning rate scheduler
cfg.lr_config = dict(policy='step', step=[1])

# Modify runtime setting
cfg.runner = dict(type='EpochBasedRunner', max_epochs=2)

# Specify the work directory
cfg.work_dir = './work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab'

# Output logs for every 10 iterations
cfg.log_config.interval = 10

# Set the random seed and enable the deterministic option of cuDNN
# to keep the results' reproducible.
from mmselfsup.apis import set_random_seed
cfg.seed = 0
set_random_seed(0, deterministic=True)

cfg.gpu_ids = range(1)

### Start self-supervised pre-train task

In [14]:
import time
import mmcv
import os.path as osp

from mmselfsup.datasets import build_dataset
from mmselfsup.models import build_algorithm
from mmselfsup.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))

# Build the algorithm
model = build_algorithm(cfg.model)
model.init_weights()

# Build the dataset
datasets = [build_dataset(cfg.data.train)]

# Start pre-train
train_model(
    model,
    datasets,
    cfg,
    distributed=False,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

/usr/local/lib/python3.7/dist-packages/mmcv/utils/registry.py:255: DeprecationWarning: The old API of register_module(module, force=False) is deprecated and will be removed, please use the new API register_module(name=None, force=False, module=None) instead.
  DeprecationWarning)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/usr/local/lib/python3.7/dist-packages/scipy/special/orthogonal.py:81: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.i

## Example to start a downstream task


In [15]:
!pwd

/content/mmselfsup


### Extract backbone weights from pre-train model

In [16]:
!python tools/model_converters/extract_backbone_weights.py \
  work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/epoch_2.pth \
  work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth

### Prepare config file

Here we create a new config file for demo dataset, actually we provided various config files in directory `configs/benchmarks`.

In [17]:
# Load the basic config file
from mmcv import Config
benchmark_cfg = Config.fromfile('configs/benchmarks/classification/imagenet/resnet50_linear-8xb32-steplr-100e_in1k.py')

# Modify the model
checkpoint_file = 'work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth'
# Or directly using pre-train model provided by us
# checkpoint_file = 'https://download.openmmlab.com/mmselfsup/moco/mocov2_resnet50_8xb32-coslr-200e_in1k_20220225-89e03af4.pth'

benchmark_cfg.model.backbone.frozen_stages=4
benchmark_cfg.model.backbone.init_cfg = dict(type='Pretrained', checkpoint=checkpoint_file)

# As the imagenet_examples dataset folder doesn't have val dataset
# Modify the path and meta files of validation dataset
benchmark_cfg.data.val.data_source.data_prefix = 'data/imagenet/train'
benchmark_cfg.data.val.data_source.ann_file = 'data/imagenet/meta/train.txt'

# Specify the learning rate scheduler
benchmark_cfg.lr_config = dict(policy='step', step=[1])

# Output logs for every 10 iterations
benchmark_cfg.log_config.interval = 10

# Modify runtime settings for demo
benchmark_cfg.runner = dict(type='EpochBasedRunner', max_epochs=2)

# Specify the work directory
benchmark_cfg.work_dir = './work_dirs/benchmarks/classification/imagenet/resnet50_8xb32-steplr-100e_in1k_colab'

# Set the random seed and enable the deterministic option of cuDNN
# to keep the results' reproducible.
from mmselfsup.apis import set_random_seed
benchmark_cfg.seed = 0
set_random_seed(0, deterministic=True)

benchmark_cfg.gpu_ids = range(1)


### Load extracted backbone weights to start a downstream task

In [18]:
import time
import mmcv
import os.path as osp

from mmselfsup.datasets import build_dataset
from mmselfsup.models import build_algorithm
from mmselfsup.apis import train_model

# Create the work directory
mmcv.mkdir_or_exist(osp.abspath(benchmark_cfg.work_dir))

# Build the algorithm
model = build_algorithm(benchmark_cfg.model)
model.init_weights()

# Build the dataset
datasets = [build_dataset(benchmark_cfg.data.train)]

# Start linear probing
train_model(
    model,
    datasets,
    benchmark_cfg,
    distributed=False,
    timestamp=time.strftime('%Y%m%d_%H%M%S', time.localtime()),
    meta=dict())

2022-03-25 05:25:22,114 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth'}
2022-03-25 05:25:22,116 - mmcv - INFO - load model from: work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth
2022-03-25 05:25:22,122 - mmcv - INFO - load checkpoint from local path: work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth
2022-03-25 05:25:22,264 - mmcv - INFO - initialize ClsHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'layer': 'Linear'}, {'type': 'Constant', 'val': 1, 'layer': ['_BatchNorm', 'GroupNorm']}]
2022-03-25 05:25:22,285 - mmcv - INFO - 
backbone.conv1.weight - torch.Size([64, 3, 7, 7]): 
PretrainedInit: load from work_dirs/selfsup/relative-loc_resnet50_8xb64-steplr-70e_in1k_colab/relative-loc_backbone-weights.pth 
 
2022-03-25 05:25:22,287 - mmcv -

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 41/41, 1.9 task/s, elapsed: 21s, ETA:     0s

2022-03-25 05:26:04,802 - mmselfsup - INFO - head4_top1: 100.000
2022-03-25 05:26:04,804 - mmselfsup - INFO - head4_top5: 100.000
2022-03-25 05:26:04,807 - mmselfsup - INFO - Epoch(val) [1][41]	head4_top1: 100.0000, head4_top5: 100.0000
2022-03-25 05:26:12,294 - mmselfsup - INFO - Epoch [2][10/41]	lr: 3.000e+00, eta: 0:00:16, time: 0.748, data_time: 0.482, memory: 7428, loss: 0.0000, acc: 100.0000
2022-03-25 05:26:16,236 - mmselfsup - INFO - Epoch [2][20/41]	lr: 3.000e+00, eta: 0:00:10, time: 0.394, data_time: 0.148, memory: 7428, loss: 0.0000, acc: 100.0000
2022-03-25 05:26:20,883 - mmselfsup - INFO - Epoch [2][30/41]	lr: 3.000e+00, eta: 0:00:05, time: 0.465, data_time: 0.215, memory: 7428, loss: 0.0000, acc: 100.0000
2022-03-25 05:26:24,566 - mmselfsup - INFO - Epoch [2][40/41]	lr: 3.000e+00, eta: 0:00:00, time: 0.368, data_time: 0.124, memory: 7428, loss: 0.0000, acc: 100.0000
2022-03-25 05:26:24,736 - mmselfsup - INFO - Saving checkpoint at 2 epochs


[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>] 41/41, 2.1 task/s, elapsed: 20s, ETA:     0s

2022-03-25 05:26:45,476 - mmselfsup - INFO - head4_top1: 100.000
2022-03-25 05:26:45,477 - mmselfsup - INFO - head4_top5: 100.000
2022-03-25 05:26:45,480 - mmselfsup - INFO - Epoch(val) [2][41]	head4_top1: 100.0000, head4_top5: 100.0000


**Note: As the demo only has one class in dataset, the model collapsed and the results of loss and acc should be ignored.**